In [40]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [41]:
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]
y = (iris["target"] == 2).astype(np.float64)

linsvm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])

linsvm_clf.fit(X, y)
y_predict = linsvm_clf.predict([[5.5, 1.7], [6.0, 0.02]])
print(y_predict)


[1. 0.]


In [42]:
from sklearn.svm import SVC

X = iris["data"][:, (2, 3)]
y = (iris["target"] == 2).astype(np.float64)

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", SVC(kernel="linear", C=1)),
])
svm_clf.fit(X, y)
y_predict = svm_clf.predict([[5.5, 1.7], [6.0, 0.02]])
print(y_predict)


[1. 0.]


In [43]:
from sklearn.linear_model import SGDClassifier

X = iris["data"][:, (2, 3)]
y = (iris["target"] == 2).astype(np.float64)

sgd_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", SGDClassifier(loss="hinge", alpha=0.017, max_iter=1000, tol=1e-3)),
])
sgd_clf.fit(X, y.ravel())
y_predict = sgd_clf.predict([[5.5, 1.7], [6.0, 0.02]])
print(y_predict)


[1. 0.]


In [47]:
from sklearn.base import BaseEstimator


class MyLinearSVC(BaseEstimator):
    def __init__(self, C=1, eta0=1, eta_d=10000, n_epochs=1000, random_state=None):
        self.C = C
        self.eta0 = eta0
        self.n_epochs = n_epochs
        self.random_state = random_state
        self.eta_d = eta_d

    def eta(self, epoch):
        return self.eta0 / (epoch + self.eta_d)

    def fit(self, X, y):
        # Random initialization
        if self.random_state:
            np.random.seed(self.random_state)
        w = np.random.randn(X.shape[1], 1)  # n feature weights
        b = 0

        t = y * 2 - 1  # -1 if y==0, +1 if y==1
        X_t = X * t
        self.Js = []

        # Training
        for epoch in range(self.n_epochs):
            support_vectors_idx = (X_t.dot(w) + t * b < 1).ravel()
            X_t_sv = X_t[support_vectors_idx]
            t_sv = t[support_vectors_idx]

            # hinge loss
            J = 1/2 * np.sum(w * w) + self.C * \
                (np.sum(1 - X_t_sv.dot(w)) - b * np.sum(t_sv))
            self.Js.append(J)
            w_gradient_vector = w - self.C * \
                np.sum(X_t_sv, axis=0).reshape(-1, 1)
            b_derivative = -self.C * np.sum(t_sv)

            w = w - self.eta(epoch) * w_gradient_vector
            b = b - self.eta(epoch) * b_derivative

        self.intercept_ = np.array([b])
        self.coef_ = np.array([w])
        support_vectors_idx = (X_t.dot(w) + t * b < 1).ravel()
        self.support_vectors_ = X[support_vectors_idx]
        return self

    def decision_function(self, X):
        return X.dot(self.coef_[0]) + self.intercept_[0]

    def predict(self, X):
        return (self.decision_function(X) >= 0).astype(np.float64)


# Training set
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = (iris["target"] == 2).astype(np.float64).reshape(-1, 1)  # Iris virginica

mysvm_clf = MyLinearSVC(C=2, eta0=10, eta_d=1000, n_epochs=60000, random_state=2)
mysvm_clf.fit(X, y)
mysvm_clf.predict(np.array([[5.5, 1.7], [6.0, 0.02]]))


array([[1.],
       [0.]])